In [ ]:
# !pip install opentelemetry-api
# !pip install opentelemetry-sdk

In [ ]:
from pathlib import Path
import sys

# Add base dir to path to be able to access test folder.
base_dir = Path().cwd().parent.parent.resolve()
if str(base_dir) not in sys.path:
    print(f"Adding {base_dir} to sys.path")
    sys.path.append(str(base_dir))

In [ ]:
from trulens.apps.custom import instrument
from trulens.experimental.otel_tracing.core.instrument import (
    instrument as otel_instrument,
)


class TestApp:
    @instrument
    def respond_to_query(self, query: str) -> str:
        return f"answer: {self.nested(query)}"

    @instrument
    def nested(self, query: str) -> str:
        return f"nested: {self.nested2(query)}"

    @instrument
    @otel_instrument()
    def nested2(self, query: str) -> str:
        return f"nested2: {query}"

In [ ]:
from trulens.core.session import TruSession
from trulens.experimental.otel_tracing.core.init import init

session = TruSession()
session.reset_database()
init(session, debug=True)

In [ ]:
from trulens.apps.custom import TruCustomApp

test_app = TestApp()
custom_app = TruCustomApp(test_app)

with custom_app as recording:
    test_app.nested2("test")